In [1]:
!pip install efficientnet_pytorch codecarbon


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.7/516.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 6.3 MB/s eta 0:00:00
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=b24bfe1990e34a28b5b8713ad367ba4ef80c8ce819d5af932872893e4250f2e6
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.5.0
    Uninstalling termcolor-2.5.0:
      Successfully uninstalled termcolor-2.5.0
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR:

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
from codecarbon import EmissionsTracker  # For tracking CO₂ emissions
import time

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

num_epochs = 20  # Change if needed
batch_size = 32  # Adjust for Kaggle GPU memory
learning_rate = 1e-3  # Initial learning rate
image_size = 224  # EfficientNet requires 224x224 input images



Using device: cuda


In [3]:
# Define Transformations
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),  # Resize CIFAR-100 images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])  # CIFAR-100 normalization
])

# Load CIFAR-100 Dataset
train_dataset = torchvision.datasets.CIFAR100(root="./data", train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR100(root="./data", train=False, download=True, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print("Dataset Loaded Successfully ")

# Load EfficientNet-B4 with adjusted output layer for 100 classes
model = EfficientNet.from_pretrained("efficientnet-b4", num_classes=100)
model.to(device)
print("Model Loaded ")


100%|██████████| 169M/169M [00:02<00:00, 62.9MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Dataset Loaded Successfully ✅


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b4-6ed6700e.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b4-6ed6700e.pth
100%|██████████| 74.4M/74.4M [00:00<00:00, 262MB/s]


Loaded pretrained weights for efficientnet-b4
Model Loaded ✅


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Learning rate scheduler to adjust learning rate when validation loss plateaus
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=3, verbose=True)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
tracker = EmissionsTracker(project_name="EfficientNet_CIFAR100_Kaggle")
tracker.start()
print("Tracking CO₂ Emissions... ")


[codecarbon INFO @ 09:23:42] [setup] RAM Tracking...
[codecarbon INFO @ 09:23:42] [setup] CPU Tracking...
[codecarbon WARNING @ 09:23:42] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 09:23:44] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 09:23:44] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 09:23:44] [setup] GPU Tracking...
[codecarbon INFO @ 09:23:44] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 09:23:44] >>> Tracker's metadata:
[codecarbon INFO @ 09:23:44]   Platform system: Linux-6.6.56+-x86_64-with-glibc2.35
[codecarbon INFO @ 09:23:44]   Python version: 3.10.12
[codecarbon INFO @ 09:23:44]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 09:23:44]   Available RAM : 31.351 GB
[codecarbon INFO @ 09:23:44]   CPU count: 4
[codeca

Tracking CO₂ Emissions... 🌱


In [6]:
# Mixed Precision Training (AMP)
scaler = torch.cuda.amp.GradScaler()

def train_epoch(epoch, pbar):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        
        # Mixed precision training
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, targets)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        # Update progress bar with current batch progress
        pbar.update(1)
    
    epoch_loss = running_loss / total
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


def evaluate():
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, targets)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    loss = running_loss / total
    acc = 100. * correct / total
    return loss, acc


<ipython-input-6-eb05a3b7b51f>:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [7]:
from tqdm import tqdm  # Import tqdm for progress bar

start_time = time.time()  # Track total training time

for epoch in range(num_epochs):
    epoch_start = time.time()  # Track epoch start time
    
    # Create a tqdm progress bar for training
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
        train_loss, train_acc = train_epoch(epoch, pbar)  # Pass progress bar to function
    
    val_loss, val_acc = evaluate()  # Validation after each epoch
    
    epoch_time = time.time() - epoch_start  # Calculate epoch duration
    
    # Adjust Learning Rate
    scheduler.step(val_loss)
    
    print(f"\nEpoch [{epoch+1}/{num_epochs}] | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | "
          f"Epoch Time: {epoch_time:.2f} sec")
    
total_time = time.time() - start_time  # Total training duration
print(f"\nTotal Training Time: {total_time/60:.2f} minutes")



Epoch 1/20:   0%|          | 0/1563 [00:00<?, ?batch/s]<ipython-input-6-eb05a3b7b51f>:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/20:   3%|▎         | 46/1563 [00:14<07:03,  3.58batch/s][codecarbon INFO @ 09:24:02] Energy consumed for RAM : 0.000049 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 09:24:02] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:24:02] Energy consumed for all GPUs : 0.000301 kWh. Total GPU Power : 72.18080850486174 W
[codecarbon INFO @ 09:24:02] 0.000527 kWh of electricity used since the beginning.
Epoch 1/20:   6%|▋         | 99/1563 [00:29<06:53,  3.54batch/s][codecarbon INFO @ 09:24:17] Energy consumed for RAM : 0.000098 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 09:24:17] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:24:17]


Epoch [1/20] | Train Loss: 1.5016 | Train Acc: 59.10% | Val Loss: 1.0411 | Val Acc: 70.91% | Epoch Time: 499.28 sec


Epoch 2/20:   2%|▏         | 34/1563 [00:10<07:42,  3.30batch/s][codecarbon INFO @ 09:32:17] Energy consumed for RAM : 0.001664 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 09:32:17] Energy consumed for all CPUs : 0.006017 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:32:17] Energy consumed for all GPUs : 0.010680 kWh. Total GPU Power : 75.10925714120629 W
[codecarbon INFO @ 09:32:17] 0.018361 kWh of electricity used since the beginning.
Epoch 2/20:   5%|▌         | 84/1563 [00:25<07:26,  3.32batch/s][codecarbon INFO @ 09:32:32] Energy consumed for RAM : 0.001713 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 09:32:32] Energy consumed for all CPUs : 0.006194 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:32:32] Energy consumed for all GPUs : 0.010996 kWh. Total GPU Power : 75.84198930827823 W
[codecarbon INFO @ 09:32:32] 0.018903 kWh of electricity used since the beginning.
Epoch 2/20:   9%|▊         | 134/1563 [00:40<07:14,  3.29batch/s][codecarbon INF


Epoch [2/20] | Train Loss: 0.9083 | Train Acc: 73.54% | Val Loss: 0.9926 | Val Acc: 72.95% | Epoch Time: 502.96 sec


Epoch 3/20:   1%|          | 8/1563 [00:02<07:58,  3.25batch/s][codecarbon INFO @ 09:40:32] Energy consumed for RAM : 0.003280 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 09:40:32] Energy consumed for all CPUs : 0.011857 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:40:32] Energy consumed for all GPUs : 0.021031 kWh. Total GPU Power : 74.38282815302009 W
[codecarbon INFO @ 09:40:32] 0.036167 kWh of electricity used since the beginning.
Epoch 3/20:   4%|▎         | 58/1563 [00:17<07:35,  3.31batch/s][codecarbon INFO @ 09:40:47] Energy consumed for RAM : 0.003329 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 09:40:47] Energy consumed for all CPUs : 0.012034 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 09:40:47] Energy consumed for all GPUs : 0.021345 kWh. Total GPU Power : 75.49222976735265 W
[codecarbon INFO @ 09:40:47] 0.036708 kWh of electricity used since the beginning.
Epoch 3/20:   7%|▋         | 107/1563 [00:32<07:22,  3.29batch/s][codecarbon INFO

...


Epoch [18/20] | Train Loss: 0.0081 | Train Acc: 99.77% | Val Loss: 1.0058 | Val Acc: 82.85% | Epoch Time: 502.12 sec


Epoch 19/20:   2%|▏         | 31/1563 [00:09<07:42,  3.31batch/s][codecarbon INFO @ 11:54:47] Energy consumed for RAM : 0.029565 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 11:54:47] Energy consumed for all CPUs : 0.106894 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:54:47] Energy consumed for all GPUs : 0.189366 kWh. Total GPU Power : 75.17392441546792 W
[codecarbon INFO @ 11:54:47] 0.325825 kWh of electricity used since the beginning.
Epoch 19/20:   5%|▌         | 80/1563 [00:24<07:25,  3.33batch/s][codecarbon INFO @ 11:55:02] Energy consumed for RAM : 0.029614 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 11:55:02] Energy consumed for all CPUs : 0.107071 kWh. Total CPU Power : 42.5 W
Epoch 19/20:   5%|▌         | 81/1563 [00:24<07:26,  3.32batch/s][codecarbon INFO @ 11:55:02] Energy consumed for all GPUs : 0.189678 kWh. Total GPU Power : 74.82891646825802 W
[codecarbon INFO @ 11:55:02] 0.326363 kWh of electricity used since the beginning.
Epoch 19/20: 


Epoch [19/20] | Train Loss: 0.0079 | Train Acc: 99.77% | Val Loss: 1.0024 | Val Acc: 83.05% | Epoch Time: 502.08 sec


Epoch 20/20:   1%|          | 8/1563 [00:02<07:57,  3.26batch/s][codecarbon INFO @ 12:03:02] Energy consumed for RAM : 0.031181 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 12:03:02] Energy consumed for all CPUs : 0.112734 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:03:02] Energy consumed for all GPUs : 0.199713 kWh. Total GPU Power : 74.39442825846008 W
[codecarbon INFO @ 12:03:02] 0.343627 kWh of electricity used since the beginning.
Epoch 20/20:   4%|▎         | 57/1563 [00:17<07:36,  3.30batch/s][codecarbon INFO @ 12:03:17] Energy consumed for RAM : 0.031229 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 12:03:17] Energy consumed for all CPUs : 0.112911 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:03:17] Energy consumed for all GPUs : 0.200024 kWh. Total GPU Power : 74.72093744644187 W
[codecarbon INFO @ 12:03:17] 0.344165 kWh of electricity used since the beginning.
Epoch 20/20:   7%|▋         | 107/1563 [00:32<07:20,  3.31batch/s][codecarbon I


Epoch [20/20] | Train Loss: 0.0067 | Train Acc: 99.80% | Val Loss: 1.0065 | Val Acc: 82.81% | Epoch Time: 502.11 sec

Total Training Time: 167.58 minutes


In [8]:
# Stop CodeCarbon tracker and report total CO₂ emissions
emissions = tracker.stop()
print(f"Total CO₂ Emissions: {emissions:.6f} kg")


[codecarbon INFO @ 12:11:25] Energy consumed for RAM : 0.032821 kWh. RAM Power : 11.756441116333008 W
[codecarbon INFO @ 12:11:25] Energy consumed for all CPUs : 0.118665 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:11:25] Energy consumed for all GPUs : 0.210188 kWh. Total GPU Power : 62.39643990531986 W
[codecarbon INFO @ 12:11:25] 0.361674 kWh of electricity used since the beginning.


Total CO₂ Emissions: 0.103259 kg


/usr/local/lib/python3.10/dist-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])


In [ ]:
# Save trained model for future inference
torch.save(model.state_dict(), "efficientnet_cifar100.pth")
print("Model Saved Successfully ")


Model Saved Successfully ✅
